# Amortized Inference for a NLME Model

## Simulation & Training Phase

In [ ]:
from time import perf_counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from bayesflow.simulation import GenerativeModel
from bayesflow import diagnostics

In [ ]:
model_name = ['fröhlich-simple', 'fröhlich-detailed', 'fröhlich-sde',
              'pharmacokinetic_model', 'clairon_small_model'][2]
if model_name == 'fröhlich-detailed' or model_name == 'pharmacokinetic_model' or 'clairon' in model_name:
    # this is needed so julia is recognized correctly
    from juliacall import Main as jl

# specify which model to use
network_idx = 15
load_best_network = True

In [ ]:
# show or save plots
show_plots = True
save_plots = False
    
# training params
early_stopping = True  # if training should be stopped if no improvement on a validation set can be seen
presimulate = False  # if simulations should be generated before or while training of the neural networks
train_network = False  # if neural networks should be trained or only load (already trained) networks

## Load model

To train your own model you should define 
```
def batch_simulator(param_samples: np.ndarray, other_args) -> np.ndarray:
```
which takes in parameters in the form of (#simulations, #parameters) and a model-class of the form

```
class myModel(NlmeBaseAmortizer):
    def __init__(self, name: str = 'myModel'):
        # define names of parameters
        param_names = ['name1', 'name2']

        # define prior values (for log-parameters)
        prior_mean = np.array([0, 0])
        prior_cov = np.diag([1, 1])
        self.prior_type = 'normal'

        super().__init__(name=name,
                         param_names=param_names,
                         prior_mean=prior_mean,
                         prior_cov=prior_cov)

        self.simulator = Simulator(batch_simulator_fun=partial(batch_simulator,
                                                                  other_args=other_args))
```
where you specify your parameters, a parameter-prior on the individual level (standard is normal distribution) and connect your `batch_simulator` to the model.

In this notebook, the example models are loaded from an external file, where each model defines its own `batch_simulator` function, which is used to generate simulations and some other further helper function to generate plots etc.

In [ ]:
presimulation_path = '../data/'

if model_name == 'fröhlich-simple':
    from models.froehlich_model_simple import FroehlichModelSimple
    model = FroehlichModelSimple(network_idx=network_idx, load_best=load_best_network)
    
    use_presimulation = False
elif model_name == 'fröhlich-detailed':
    from models.froehlich_model_detailed import FroehlichModelDetailed
    model = FroehlichModelDetailed(network_idx=network_idx, load_best=load_best_network)

    use_presimulation = True
    presimulation_path += 'presimulations_froehlich_detailed'

elif model_name == 'fröhlich-sde':
    from models.froehlich_model_sde import FroehlichModelSDE
    model = FroehlichModelSDE(network_idx=network_idx, load_best=load_best_network)

    use_presimulation = True
    presimulation_path += 'presimulations_froehlich_sde'
    
elif model_name == 'pharmacokinetic_model':
    from models.pharmacokinetic_model import PharmacokineticModel
    model = PharmacokineticModel(network_idx=network_idx, load_best=load_best_network)
    
    use_presimulation = True
    presimulation_path += 'presimulations_pharma'
    
elif model_name == 'clairon_small_model':
    from models.clairon_small_model import ClaironSmallModel
    model = ClaironSmallModel(network_idx=network_idx, load_best=load_best_network)

    use_presimulation = True
    presimulation_path += 'presimulations_small_clairon'
else:
    raise NotImplementedError('model not implemented')

In [ ]:
if show_plots:
    # the example models all support this function (which is optional)
    model.plot_example()

In [ ]:
# example training
max_epochs = 1
iterations_per_epoch = 10

In [ ]:
if presimulate:
    # if you want to generate simulations before training use this code block
    generative_model = GenerativeModel(model.prior, model.simulator)
    start_time = perf_counter()
    np.random.seed(42)
    generative_model.presimulate_and_save(batch_size=128, 
                                          folder_path=presimulation_path,
                                          iterations_per_epoch=iterations_per_epoch,
                                          epochs=max_epochs,
                                          disable_user_input=True)
    end_time = perf_counter()
    print(f'simulation time: {(end_time-start_time)/60} minutes')

## Training

In [ ]:
# create your neural amortizer, the class NlmeBaseAmortizer predefines a network
# if you want to change the network architecture, check out the examples in the function 'load_amortizer_configuration' in one of the models classes 
# you can change summary and inference network to a wide range of architectures
trainer = model.build_trainer('../networks/' + model.network_name)
print(model.network_name)

each epoch a number of iterations each with batch_size many simulations is performed

In [ ]:
history = {}
if train_network and not use_presimulation:
    # online training: simulation is done whenever needed
    start_time = perf_counter()
    history = trainer.train_online(epochs=max_epochs,
                                  iterations_per_epoch=iterations_per_epoch,
                                  batch_size=128,
                                  early_stopping=early_stopping,
                                  validation_sims=iterations_per_epoch)
    end_time = perf_counter()
    print(f'training time: {(end_time-start_time)/60} minutes')

In [ ]:
if train_network and use_presimulation:
    # prepare optimizer for offline training (simulations were done before training)
    trainer._setup_optimizer(optimizer=None,
                             epochs=max_epochs,
                             iterations_per_epoch=iterations_per_epoch)

In [ ]:
if train_network and use_presimulation:
    # simulation done before, start training now
    start_time = perf_counter()
    history = trainer.train_from_presimulation(presimulation_path=presimulation_path,
                                               optimizer=trainer.optimizer,
                                               max_epochs=max_epochs,
                                               early_stopping=early_stopping,
                                               validation_sims=iterations_per_epoch)

    end_time = perf_counter()
    print(f'training time: {(end_time-start_time)/60} minutes')

In [ ]:
# load or save losses
if train_network:
    history['train_losses'].to_csv(f"../networks/losses/{model.network_name}_train_losses.csv")
    history['val_losses'].to_csv(f"../networks/losses/{model.network_name}_val_losses.csv")
else:
    train_losses = pd.read_csv(f"../networks/losses/{model.network_name}_train_losses.csv", index_col=0)
    val_losses = pd.read_csv(f"../networks/losses/{model.network_name}_val_losses.csv", index_col=0)
    history = {'train_losses': train_losses,
              'val_losses': val_losses}

In [ ]:
# show training loss
fig_losses = diagnostics.plot_losses(history['train_losses'], val_losses=history['val_losses'],
                                     moving_average=True)
if save_plots:
    fig_losses.savefig(f'../plots/calibration/{model_name}/{model.network_name}_fig_losses.png')

In [ ]:
# load real data (here for the first time), simulate posterior and plot
test_data = model.load_data(n_data=25)  # or load synthetic data with synthetic=True
data_posterior_draws = model.draw_posterior_samples(data=test_data, n_samples=100)

rows = 5
fig, ax = plt.subplots(rows, int(np.ceil(len(test_data) / rows)), tight_layout=True, 
                       figsize=(10, rows*3), sharey='row')
axis = ax.flatten()
    
for p_id in tqdm(range(len(test_data))):
    axis[p_id] = model.prepare_plotting(test_data[p_id], data_posterior_draws[p_id], axis[p_id])
    _, labels = axis[p_id].get_legend_handles_labels()
    axis[p_id].set_title(f'Individual {p_id}')
    if p_id % rows != 0:
        axis[p_id].set_ylabel('')

fig.legend(labels, ncol=3, loc='upper center', bbox_to_anchor=(0.5, -0.01))
if save_plots:
    fig.savefig(f'../plots/calibration/{model_name}/{model.network_name}_fit.png')
plt.show()

## Validate Posteriors using SBC-Plots

To check if the training converged, you should validate the posteriors using the following visual diagnostics.
If you want, you can also compare the posteriors to any other method (like MCMC or ABC). Check out the "Validate Posterior with MCMC"-notebook in the Experiments folder for a comparison to MCMC.

In [ ]:
new_sims = model.generate_simulations_from_prior(trainer=trainer, n_samples=2500)
posterior_draws = model.draw_posterior_samples(data=new_sims['summary_conditions'], n_samples=100)

In [ ]:
fig_sbc = diagnostics.plot_sbc_histograms(post_samples=posterior_draws,
                                          prior_samples=new_sims['parameters'],
                                          param_names=model.log_param_names)

for _ax in fig_sbc.axes[model.n_params:]:  
    _ax.remove()  # TODO: this is a bug in BayesFlow, figure should be cleaned already
if save_plots:
    fig_sbc.savefig(f'../plots/calibration/{model_name}/{model.network_name}_fig_sbc.png')

In [ ]:
fig_ecdf_diff = diagnostics.plot_sbc_ecdf(post_samples=posterior_draws,
                                          prior_samples=new_sims['parameters'],
                                          difference=True,
                                          stacked=False,
                                          param_names=model.log_param_names)

if save_plots:
    fig_ecdf_diff.savefig(f'../plots/calibration/{model_name}/{model.network_name}_fig_ecdf_diff.png')

In [ ]:
fig_posterior = diagnostics.plot_posterior_2d(posterior_draws=posterior_draws[0],
                                              prior=model.prior,
                                              param_names=model.log_param_names)

if save_plots:
    fig_posterior.savefig(f'../plots/calibration/{model_name}/{model.network_name}_fig_posterior.png')

### Get SBC Plots for Different Epochs

In [ ]:
# if checkpoints are saved during optimization one can look at earlier calibration plots
ckpt_list = []# [1, 10, 100, 200, 300, 400, 500]

In [ ]:
for ckpt in ckpt_list:
    print('Load checkpoint', ckpt)
    # restore checkpoint
    trainer.checkpoint.restore('../networks/' + model.network_name + f'/ckpt-{ckpt}')
    # sample posterior
    posterior_draws = model.draw_posterior_samples(data=new_sims['summary_conditions'], n_samples=250)

    # make sbc plots
    fig_ecdf_diff = diagnostics.plot_sbc_ecdf(post_samples=posterior_draws,
                                              prior_samples=new_sims['parameters'],
                                              difference=True,
                                              stacked=False,
                                              param_names=model.log_param_names)
    if save_plots:
        fig_ecdf_diff.savefig(f'../plots/calibration/epochs_compare/{model.network_name}_fig_ecdf_diff_{ckpt}_.png')
    plt.show()